# 2023-2 언어데이터과학 24강 (2023-11-29) 실습 (1) `gensim` 패키지와 Word2Vec 모델 훈련

## 오늘의 목표

1. 모두의 말뭉치 '온라인 대화 말뭉치'의 발화 자료로 Word2Vec Skip-gram 모델을 훈련시킬 수 있다.
2. 단어 사이의 의미 유사성을 벡터들의 코사인 유사도로 계량화하여 설명할 수 있다.
3. t-SNE 기법을 사용하여 단어 벡터들을 2차원 평면에 시각화할 수 있다.
4. Word2Vec 모델을 사용하여 연령대에 따른 단어의 분포 변화를 추적할 수 있다.

## 0. 준비

### 파이썬 모듈 설치하기

Python에서 Word2Vec 모델을 사용하기 위해서는 `gensim` 모듈이 필요하다.

In [1]:
%pip install gensim

Note: you may need to restart the kernel to use updated packages.


## 1. 데이터 가공

12–15강에서 만든 모두의 말뭉치 [온라인 대화 말뭉치] 파일을 읽고 `gensim` 모듈에서 사용 가능한 코퍼스로 가공하자.

In [2]:
import pandas as pd
from gensim.models import Word2Vec
from tqdm import tqdm

### 데이터 파일 읽기

In [3]:
DATA_PATH = '../data/NIKL_OM_form_age_sex.csv.tar.gz'

In [4]:
utterances = pd.read_csv(DATA_PATH, compression='gzip', on_bad_lines='skip')
utterances

,data/NIKL_OM_form_age_sex.csv,form,speaker_id,age,sex
0,MDRW2100000001.1.1,안녕하세요,MDRW2100000001_1,20대,여성
1,MDRW2100000001.1.4,이거 해봐요><,MDRW2100000001_1,20대,여성
2,MDRW2100000001.1.7,오 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,MDRW2100000001_1,20대,여성
3,MDRW2100000001.1.8,안챙겨도 잘커요,MDRW2100000001_1,20대,여성
4,MDRW2100000001.1.9,너무 맞는데요ㅜㅜ?,MDRW2100000001_1,20대,여성
...,...,...,...,...,...
2977836,MMRW2100000241.1.2775,한 번도 안 써봄...?,MMRW2100000241_2,20대,여성
2977837,MMRW2100000241.1.2776,그거 개꿀인디,MMRW2100000241_2,20대,여성
2977838,MMRW2100000241.1.2780,ㅋㅋㅋㅋㅋㅋ잠수복 개귀여웤ㅋㅋㅋㅋ,MMRW2100000241_2,20대,여성
2977839,MMRW2100000241.1.2786,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ가즈아!,MMRW2100000241_2,20대,여성


In [5]:
utterances = pd.read_csv(DATA_PATH, compression='gzip', on_bad_lines='skip')
utterances.dropna(inplace=True)
utterances.rename(columns={utterances.columns[0]: 'id'}, inplace=True)
utterances.set_index('id', inplace=True)
utterances

,form,speaker_id,age,sex
id,,,,
MDRW2100000001.1.1,안녕하세요,MDRW2100000001_1,20대,여성
MDRW2100000001.1.4,이거 해봐요><,MDRW2100000001_1,20대,여성
MDRW2100000001.1.7,오 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,MDRW2100000001_1,20대,여성
MDRW2100000001.1.8,안챙겨도 잘커요,MDRW2100000001_1,20대,여성
MDRW2100000001.1.9,너무 맞는데요ㅜㅜ?,MDRW2100000001_1,20대,여성
...,...,...,...,...
MMRW2100000241.1.2774,그 낚시대회 전용 투망 있을걸???,MMRW2100000241_2,20대,여성
MMRW2100000241.1.2775,한 번도 안 써봄...?,MMRW2100000241_2,20대,여성
MMRW2100000241.1.2776,그거 개꿀인디,MMRW2100000241_2,20대,여성


### 데이터 형변환

일반적으로 Python의 여러 라이브러리에서 코퍼스를 다룰 때는 한 문장을 단어들의 리스트로 표현하고, 코퍼스 전체를 문장들의 리스트로 표현한다.

지금 가지고 있는 데이터프레임에서는 문장에 해당하는 발화가 `str` 자료형이므로, `str.split()` 메소드를 사용하여 단어들의 리스트로 만들어 주자.

In [6]:
corpus = utterances['form'] # edit this line
print(corpus[:5])

id
MDRW2100000001.1.1            [안녕하세요]
MDRW2100000001.1.4        [이거, 해봐요><]
MDRW2100000001.1.7    [오, ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ]
MDRW2100000001.1.8        [안챙겨도, 잘커요]
MDRW2100000001.1.9      [너무, 맞는데요ㅜㅜ?]
Name: form, dtype: object


## 2. Word2Vec 모델 훈련

### 모델 초기화

In [7]:
model = Word2Vec(
    sg=1,
    min_count=5,
    vector_size=100,
    window=2,
    negative=5,
)

### Vocabulary 구축

In [8]:
model.build_vocab(corpus_iterable=tqdm(corpus))

  0%|          | 0/2977840 [00:00<?, ?it/s]

100%|██████████| 2977840/2977840 [00:05<00:00, 575917.00it/s]


### 모델 훈련시키기

In [9]:
model.train(
    corpus_iterable=corpus,
    total_examples=model.corpus_count,
    epochs=5
)
# (trained_word_count, raw_word_count)

(32843899, 46433495)

### 훈련 결과

Vocabulary 확인

In [10]:
# 빈도순 상위 30개 단어
print(model.wv.index_to_key[:30])

['아', 'ㅋㅋ', 'ㅋㅋㅋ', '저는', '진짜', '근데', '너무', 'ㅎㅎ', '다', '저도', '좀', 'ㅋㅋㅋㅋ', '잘', '그', '나', '네', '더', '많이', '전', '그냥', '오', '난', 'ㅠㅠ', '맞아요', 'ㅋㅋㅋㅋㅋ', '안', '오늘', '저', '그래서', '그럼']


단어 벡터 확인

In [11]:
print(model.wv['ㅋㅋ'])

[ 0.15738676  0.24605632 -0.43182623  0.10587132 -0.35331696 -0.33401868
  0.2000567   0.05408589 -0.13641225 -0.24742404  0.19703132 -0.15992947
  0.02313019  0.28577062 -0.26262215  0.01126668  0.09560379  0.11013892
  0.210887   -0.44798023  1.1004206   0.36859006  0.08071067 -0.30098853
  0.24289334 -0.7887884   0.34335396 -0.01580519  0.2203314   0.6322042
  0.10620683  0.00429792  0.44287267 -0.08426949 -0.24569242 -0.26391363
  0.06864527 -0.5362569  -0.10555842 -1.0280269  -0.3204698  -0.2898516
 -0.07682578 -0.25533712  0.39715    -0.14620118 -0.15547433 -0.85613203
  0.9472959   0.486032    0.54929185 -0.42307407 -0.12352052 -0.39435896
 -0.43858418  0.6094522   0.36039293  1.0375626  -0.21397994  0.16216378
  0.16238384 -0.47523463  0.27717376  0.33243933 -0.38155496 -0.3853603
  0.15183286  0.07056238 -1.2693357   0.02403639  0.20827736 -0.07857645
  0.14170831  0.72060084 -0.55891794  0.28072774 -0.14438684 -0.06744284
 -0.6136532  -0.34331676 -0.15361954  0.19866726 -0.09

## 3. Word2Vec 모델 활용

### `gensim`의 주요 기능



코사인 유사도 계산

In [12]:
print(model.wv.similarity(w1='펜', w2='연필'))
print(model.wv.similarity(w1='펜', w2='좋아요'))

0.8882772
0.37515274


평행사변형 모형

In [13]:
# 아빠 : 엄마 = X : 할머니
# 아빠 - 엄마 = X - 할머니
# X = 할머니 + 아빠 - 엄마
print() # ADD THIS LINE

[('할아버지', 0.8357796669006348), ('형부', 0.7990549206733704), ('니네', 0.7917910218238831), ('오빠랑', 0.7864155173301697), ('이모가', 0.7850600481033325), ('name10', 0.7814578413963318), ('할머니랑', 0.7792093753814697), ('너희', 0.7789581418037415), ('등록금', 0.7778154611587524), ('아빠랑', 0.7772243022918701)]


우리 중에 스파이가 있는 것 같아

In [14]:
print(model.wv.doesnt_match(['ㅋㅋ', 'ㅎㅎ', '^^', 'ㅠㅠ']))

ㅠㅠ


### '완전'과 '아주'의 이웃 비교

In [15]:
model.wv.most_similar(['완전'])

[('짱', 0.7017574906349182),
 ('되게', 0.671169638633728),
 ('디게', 0.641437828540802),
 ('대박', 0.628450870513916),
 ('넘나', 0.6200318336486816),
 ('아주', 0.6100650429725647),
 ('왕', 0.6088557839393616),
 ('무지', 0.6022351384162903),
 ('연기', 0.6018117666244507),
 ('너뮤', 0.5967180728912354)]

In [16]:
model.wv.most_similar(['아주'])

[('매우', 0.7364714741706848),
 ('굉장히', 0.6915157437324524),
 ('무척', 0.6906196475028992),
 ('상당히', 0.6799764633178711),
 ('되게', 0.6653771996498108),
 ('무지', 0.6599602699279785),
 ('디게', 0.6431306600570679),
 ('정말정말', 0.6371215581893921),
 ('꽤나', 0.6204665303230286),
 ('넘나', 0.6162236332893372)]

### 모델 저장하기

In [17]:
model.save('../models/word2vec-modu-online')

: 